In [267]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
from sklearn.model_selection import train_test_split
from datasets import load_dataset

ImportError: cannot import name 'is_exception' from 'fsspec.utils' (/home/zum/anaconda3/lib/python3.8/site-packages/fsspec/utils.py)

In [234]:
datapath = './data/tweet_emotions.csv'

data = pd.read_csv(datapath)
process_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_url =  "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2"

In [235]:
print(set(data['sentiment'].values))

{'hate', 'neutral', 'enthusiasm', 'empty', 'surprise', 'love', 'happiness', 'boredom', 'relief', 'anger', 'fun', 'worry', 'sadness'}


In [239]:
data['Labels'] = data['sentiment'].map({'love': 0,
                                       'boredom': 7,
                                       'worry': 2,
                                       'sadness': 3,
                                       'relief': 4,
                                       'neutral': 5,
                                       'enthusiasm': 6,
                                       'empty': 7,
                                       'anger': 9,
                                       'hate': 9,
                                       'fun': 10,
                                       'happiness': 1,
                                       'surprise': 8})

In [240]:
classes = set(data['Labels'].values)

In [241]:
for sent in classes:
    print("{} : {}".format(sent, len(data[data['Labels'] == sent])))

0 : 3842
1 : 5209
2 : 8459
3 : 5165
4 : 1526
5 : 8638
6 : 759
7 : 1006
8 : 2187
9 : 1433
10 : 1776


In [242]:
num_classes= len(data['Labels'].value_counts())

In [243]:
print(set(data['Labels'].values))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


In [244]:
print(data.columns)

Index(['tweet_id', 'sentiment', 'content', 'Labels'], dtype='object')


In [245]:
data = data.drop(['tweet_id'], axis=1)
data = data.drop(['sentiment'], axis=1)

In [246]:
print(data.columns)

Index(['content', 'Labels'], dtype='object')


In [247]:
y = tf.keras.utils.to_categorical(data['Labels'].values, num_classes=num_classes)
x_train, x_test, y_train, y_test = train_test_split(data['content'], y, test_size=0.15)

In [248]:
preprocessor = hub.KerasLayer(process_url)

encoder = hub.KerasLayer(bert_url)

In [249]:
test_text = ['an exmaple of a tweet!?']

processed = preprocessor(test_text)
results = encoder(processed)

In [250]:
def build_emotion_model():
    input_layer = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    net = preprocessor(input_layer)
    net = encoder(net)
    net = tf.keras.layers.Dropout(0.2, name="dropout")(net['pooled_output'])
    net = tf.keras.layers.Dense(num_classes, activation='softmax', name='output')(net)
    return tf.keras.Model(input_layer, net)

In [251]:
emotion_model = build_emotion_model()

In [252]:
raw_result = emotion_model(tf.constant(test_text))
print(raw_result)

tf.Tensor(
[[0.07121565 0.04827434 0.0162382  0.06660061 0.03382417 0.21303013
  0.0569303  0.07602425 0.13863552 0.03867605 0.24055083]], shape=(1, 11), dtype=float32)


In [253]:
emotion_model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_21 (KerasLayer)    {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                   

In [254]:
metrics = [tf.metrics.CategoricalAccuracy(name="accuracy")]

In [255]:
emotion_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=metrics)

In [256]:
history = emotion_model.fit(x_train, y_train, epochs=3,
                           validation_data = (x_test, y_test))

Epoch 1/3
1063/1063 [==============================] - 428s 400ms/step - loss: 2.1340 - accuracy: 0.2520 - val_loss: 2.0078 - val_accuracy: 0.2977
Epoch 2/3
1063/1063 [==============================] - 440s 414ms/step - loss: 2.0459 - accuracy: 0.2824 - val_loss: 2.0081 - val_accuracy: 0.2983
Epoch 3/3
1063/1063 [==============================] - 441s 415ms/step - loss: 2.0176 - accuracy: 0.2943 - val_loss: 1.9855 - val_accuracy: 0.3157


In [263]:
emotion_model(tf.constant(['that makes me feel elated']))

<tf.Tensor: shape=(1, 11), dtype=float32, numpy=
array([[0.04596546, 0.04604487, 0.3074423 , 0.20900224, 0.01844118,
        0.16234991, 0.00957326, 0.03586677, 0.06250266, 0.08517534,
        0.01763606]], dtype=float32)>